### 문제
- 길이 N의 리스트안에 각 원소마다 초기값이 있고, 1번 쿼리가 주워질때마다 등차수열의 값을 증가시키는 문제이다.

### 아이디어
- 각 원소에 더해야 하는 등차수열의 공식은 바뀌지 않는다. 
  - 즉, 시작 위치와 그 길이를 안다면 원소의 값을 구하는 것은 나중에 한꺼번에 처리할 수 있을 것이다. 일종의 오프라인 쿼리 아이디어로.
  - DP[i] = `i번째 인덱스에서 처리된 1번 쿼리의 번호`

In [ ]:
import io, os, sys
input=io.BytesIO(os.read(0,os.fstat(0).st_size)).readline

def sol():
  N = int(input())
  L = [*map(int, input().split())]
  
  M = int(input())
  Queries = []
  ans = []
  DP = [0] * N #DP[i] i번째 인덱스에서 가장 최근에 처리된 쿼리의 인덱스
  for i in range(M) :
    q, *l = map(int, input().split())
    if q == 1 :
      Queries.append((i, l[0]-1, l[1]-1))
    elif q == 2 :
      x = l[0] - 1
      Queries.append(None)
      for i in range(DP[x], len(Queries)) :
        if not Queries[i]: continue
        _, s, e = Queries[i]
        if not (s <= x <= e) : continue #해당 쿼리가 범위안에 없으면 무시
        L[x] += x - s + 1

      DP[x] = i #현재 쿼리가 갱신되었다.
      ans.append(L[x])
  sys.stdout.write("\n".join(map(str, ans)))

sol()

- TLE. 역시 쿼리 처리가 $O(N)$ 이면 뚫을 수가 없다.
  - 5만번의 1번쿼리 이후 5만번의 2번쿼리를 하도록 하면 최악케이스가 만들어진다. 이때 25억번 연산하게 되므로 TLE.

### 아이디어2
- lazy로 접근해볼 수 있을수도 있을 것 같다.
  - 태그를 까니 lazy, segment로, lazy가 맞다. 어떻게 할 수 있을까?
  - 우선 1번 쿼리를 모두 포함하는 부모 노드에는 등차수열의 합을 둘 수 있을 것 같다. 중요한건 수열이 어디서 시작했고, 어디서 끝났는지의 정보인데..
  - 세그먼트 트리하나를 등차수열의 합을 구하는 용도로 쓸 수 있을 것 같다. 예를들어 3 ~ 8까지의 합을 빠르게 구할 수 있다.

### 풀이
- lazy세그로 구간에 서로 다른 숫자를 더하게 만드는 것은 힘드므로, 구간 내에 같은 숫자를 더하도록 변형할 수 있다.
  - 수열은 공차가 1인 등차 수열이므로, 수열을 변형해야 한다. \
  수열 $A$ 에 대한 쿼리 $Q(l, r): A_l + 1, A_{l+1} := A_{l+1} + 2, ... A_r := A_r + R-L+1, A_{r+1} := A_{r+1} + 0$ 이라고 생각해보자. \
  $A_{r+1}$ 에는 0을 더하는 것에 주의한다. 여기서 수열 $A$ 의 변형 $A'_i = A_i - A_{i-1}$ 을 생각할 수 있다.
    - 이렇게 되면, 수열 $A'$ 에 대한 쿼리 $Q'(l, r): A_l := A_l + 1, A_{l+1} := A_{l+1} + 1, ... A_r := A_r + 1$ 으로, 각 원소에 1씩 더하는 연산으로 변형할 수 있다.
    - 그러나, 저것이 끝이 아니다. 마지막 원소에 대해서는 0을 더하는 것이라고 생각하여, $A_{r+1} := 0 - A_r$ 을 추가로 더해줘야 한다.
    - 최종적으로 $Q'(l, r): A_l := A_l + 1, A_{l+1} := A_{l+1} + 1, ... A_r := A_r + 1, A_{r+1} := - A_r$ 이 된다.
  - imos의 구간 누적합을 구하는 아이디어를 이용하여, $\displaystyle Q2(X) = \sum_{i=1}^{X} Q'(i)$ 를 구하면 원하는 값을 얻을 수 있다. 
    - 원래 욱제가 세어 놨던 이미 떨어진 별들의 개수와 함께 더하면 쿼리의 정답이 될 것이다.
- 시간복잡도 $O(M \log N)$

In [ ]:
import io, os, sys
input=io.BytesIO(os.read(0,os.fstat(0).st_size)).readline

from operator import add
class LazySegmentTree:
  def __init__(self, data, f=add, fl=lambda i, x, cnt: i+x*cnt, fll=add, default=0, ldefault=0):
    self.f = f
    self.fl = fl
    self.fll = fll
    self.default = default
    self.ldefault = ldefault
    self.len = len(data)
    self.L = [default] * self.len + data
    self.lazy = [ldefault] * self.len
    self.cnt = [0] * self.len + [1] * self.len

    for i in reversed(range(self.len)):
      self.L[i] = self.f(self.L[i*2], self.L[i*2+1])
      self.cnt[i] = self.cnt[i*2] + self.cnt[i*2+1]

  def _apply(self, i) :
    if self.lazy[i] == self.ldefault: return
    self._push(i*2, self.lazy[i])
    self._push(i*2+1, self.lazy[i])
    self.lazy[i] = self.ldefault 

  def _propagate(self, i):
    for h in reversed(range(1, self.len.bit_length()+1)) :
      idx = i >> h

      if idx == self.default : continue
      self._apply(idx)

  def _push(self, idx, val) :
    self.L[idx] = self.fl(self.L[idx], val, self.cnt[idx])
    if idx < self.len :
      self.lazy[idx] = self.fll(self.lazy[idx], val)

  def _build(self, idx) :
    while idx :
      if idx < self.len :
        self._apply(idx)
        self.L[idx] = self.f(self.L[idx*2], self.L[idx*2+1])
      idx >>= 1

  def query(self, _s, _e) : #[s, e]
    _s += self.len
    _e += self.len
    s = _s
    e = _e + 1
    self._propagate(_s)
    self._propagate(_e)

    res = self.default
    while s < e :
      if s & 1 :
        res = self.f(res, self.L[s])
        s += 1
      if e & 1 :
        e -= 1
        res = self.f(res, self.L[e])
      s >>= 1
      e >>= 1
    return res

  def update(self, _s, _e, x):
    _s += self.len
    _e += self.len
    s = _s
    e = _e + 1
    self._propagate(_s)
    self._propagate(_e)

    while s < e :
      if s & 1 :
        self._push(s, x)
        s += 1
      if e & 1 :
        e -= 1
        self._push(e, x)
      s >>= 1
      e >>= 1
    
    self._build(_s)
    self._build(_e)

def sol():
  N = int(input())
  L = [*map(int, input().split())]
  
  LST = LazySegmentTree([0] * (N + 10))
  M = int(input())
  ans = []
  for _ in range(M) :
    q, *args = map(int, input().split())
    if q == 1 :
      l, r = args
      LST.update(l-1, r-1, 1)
      LST.update(r, r, -(r-l+1)) #마지막 원소는 구간 크기만큼 뺀다
    elif q == 2 :
      x = args[0]
      ans.append(LST.query(0, x-1) + L[x-1])

  sys.stdout.write('\n'.join(map(str, ans)))

sol()